# photometry_bkgS3.ipynb

全てのデータで開口測光を行う。モデル推定位置(緯度・経度)において明るさ(EFPの明るさとバックグラウンド)を測定し保存する。

In [12]:
import numpy as np
import math
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.stats import sigma_clipped_stats
from photutils.detection import DAOStarFinder
from photutils.aperture import aperture_photometry
from photutils.aperture import CircularAperture
from photutils.aperture import CircularAnnulus
from numba import njit
import os
import pandas as pd

import spiceypy as spice
import hstprojimage as hst
from TScmap import TScmap

fontname = 'Nimbus Sans'
plt.rcParams.update({'font.sans-serif': fontname,
                    'font.family': 'sans-serif',
                     'mathtext.fontset': 'custom',
                     'mathtext.rm': fontname,
                     'mathtext.it': fontname+':italic',
                     # 'mathtext.bf': 'Nimbus Sans:italic:bold',
                     'mathtext.bf': fontname+':bold'
                     })

spice.furnsh('kernel/cassMetaK.txt')

# cpalette = ['#8ecae6', '#ffb703', '#023047', '#fb8500', '#219ebc']
cpalette = ['#0F4C81', '#FF6F61', '#645394',
         '#84BD00', '#F6BE00', '#F7CAC9', '#0F80E6']

In [13]:
# Model positions
SATELLITE = ['EU']              # 'AM', 'IO', 'EU', 'GA'
model_fpN= np.loadtxt('data/JRM33/2021je007055-sup-0003-table si-s02.txt', skiprows=3)
model_fpN = np.vstack((model_fpN, model_fpN[0:2,:]))
model_fpS = np.loadtxt('data/JRM33/2021je007055-sup-0004-table si-s03.txt', skiprows=3)
model_fpS = np.vstack((model_fpS, model_fpS[0:2,:]))
satellite_name = ['AM', 'IO', 'EU', 'GA']          # 'AM', 'IO', 'EU', 'GA'
satellite_color = ['#888888', '#000000', '#f24875', '#888888']
satellite_marker = ['o', '*', 's', 'd']
satellite_index = [satellite_name.index(i) for i in (SATELLITE)]

### Functions

In [14]:
def dataload(filename, extdeg):
    """ Description

    Args:
        filename (str): fits file
        extdeg (float): extent of the image [deg]

    Returns:
        h (class): 
        dimage (np.array): polar projected image
    """

    ext = 0
    hext = 1
    with fits.open(filename) as hdul:
        image = hdul[ext].data      # .astype(np.float32)
        h = hdul[hext].header

    h = hst.HSTProjImage(filename)
    h.readHSTFile()
    h.MOON = 'EUROPA'
    hem = h.alm.hemisph
    # print('hem', hem)

    fig, ax = plt.subplots(dpi=200)
    ax, dimage, gl = h.tvPolar(ax, vmin=10, vmax=2000,
                    draw_labels=False, refmainoval=False,
                    satovals=['eu'],
                    reflon=None,
                    # reflon=h.alm.cml,
                    ext=extdeg,
                    )
    plt.close()
    return h, dimage

In [15]:
def lat_wlong2px(lat, wlong, N0px, extdeg, hem):
    """_summary_

    Args:
        lat (float): latitude of the point [deg]
        wlong (float): west longitude of the point [deg]
        N0px (int): number of pixels in an axis [px]
        extdeg (float): extent of the image [deg]

    Returns:
        X: X position [px]
        Y: Y position [px]
    """
    if hem=='south':
        lat = lat * (-1)

    elon = 360-(wlong-90)        # [deg]
    colat = 90-lat               # [deg]
    R = (N0px/2)*(np.tan(np.radians(colat/2))/np.tan(np.radians((90-extdeg)/2)))     # [px]
    X = R*np.cos(np.radians(elon))         # [px]
    Y = R*np.sin(np.radians(elon))         # [px]
    X += N0px/2
    Y += N0px/2
    return X, Y

In [16]:
def px2lat_wlong(X, Y, N0px, extdeg, hem):
    """_summary_

    Args:
        X (float): X position(s) [px]
        Y (float): Y position(s) [px]
        N0px (int): number of pixels in an axis [px]
        extdeg (float): extent of the image [deg]

    Returns:
        lat: latitude [deg]
        wlong: west longitude [deg]
    """
    posX = X-N0px/2
    posY = Y-N0px/2

    posR = np.sqrt(posX**2 + posY**2)
    poscolat = np.degrees(2*np.arctan2(posR*np.tan(0.5*np.radians(90-extdeg)), N0px/2))
    postheta = np.arctan2(posY,posX)-0.5*np.pi

    wlong = -np.degrees(postheta)
    lat =  90-poscolat

    if hem=='south':
        lat = lat*(-1)

    return lat, wlong

In [17]:
def Aperture(IMAGE, POSITION):
    """_summary_

    Args:
        IMAGE (2d ndarray): raw image
        POSITION (1d array): position of one spot in the raw image

    Returns:
        apertures (CircularAperture): \n
        rawflux (QTable): QTable including the spot position and the final photometry averaged in pixels. \n
            `['id']` \n
            `['xcenter']` : x position [px] \n
            `['ycenter']` : y position [px] \n
            `['aperture_sum']` : summation of brightness in the target aperture
            `['annulus_median']` : the median inside the donut \n
            `['aper_bkg']` : summation of the median inside the donut \n
            `['final_phot'] = rawflux['aperture_sum'] - rawflux['aper_bkg']` \n
            `['final_phot_ave'] = rawflux['final_phot']/apertures.area`
            
    """
    # Aperture photometry
    apertures = CircularAperture(POSITION, r=4.)
    rawflux = aperture_photometry(IMAGE, apertures)

    # Make a donut to calculate the local background in.
    annulus_apertures = CircularAnnulus(POSITION, r_in=5.5, r_out=9.)

    # Sigma-clipping
    # http://www.astroarts.com/products/stlimg7/support/faq-general/composite_methods-j.shtml
    annulus_masks = annulus_apertures.to_mask(method='center')

    # Mask inside the donut
    annulus_data = annulus_masks.multiply(IMAGE)
    annulus_data_1d = annulus_data[annulus_masks.data > 0]  # 輪っか内部の値だけ抽出

    # Calculate the median as the local background
    _, median_sigclip, _ = sigma_clipped_stats(annulus_data_1d)
    bkg_median = median_sigclip*1.0

    # Add results to the rawflux data
    rawflux['annulus_median'] = bkg_median
    rawflux['aper_bkg'] = bkg_median*apertures.area
    rawflux['final_phot'] = rawflux['aperture_sum'] - rawflux['aper_bkg']
    rawflux['final_phot_ave'] = rawflux['final_phot']/apertures.area
    
    return apertures, rawflux

### Code

In [19]:
extdeg = 45.0
year = '2014'
yearlydata_dir = 'data/red_bkg/'+year

doy_visit_list = sorted(os.listdir(yearlydata_dir))
for doyvisit in doy_visit_list[6:7]:
    fits30s = sorted(os.listdir(yearlydata_dir+'/'+doyvisit))
    
    savedir = 'img/bkgS3/'+'EUROPA'+'/'+year+'/'+doyvisit
    brightnessCSV = ['date', 'lat [deg]', 'wlong [deg]', 'Moon S3 [deg]', 'spot brightness [kR]', 'local background [kR]']
    brightlist = np.zeros((len(fits30s), 5)).tolist()
    
    try:
        os.makedirs(savedir)
    except FileExistsError:
        savedir += '_R'
        os.makedirs(savedir)

    for j in range(len(fits30s)):
        dataname = yearlydata_dir+'/'+doyvisit+'/'+fits30s[j]
        print(dataname)
        h, dimage = dataload(dataname, extdeg)
        hem = h.alm.hemisph

        Npx = dimage.shape[0]
        dimage = dimage[:,::-1]
        if hem == 'south':
            dimage = dimage[:,::-1]

        # JRM33 Estimation of EFP
        print('=== JRM33 Estimation ===')
        print('Wlong [deg]', h.s3wlon_lin )
        print('Lat [deg]', h.s3lat_lin)
        ftpX, ftpY = lat_wlong2px(h.s3lat_lin, h.s3wlon_lin, Npx, extdeg, hem)
        # print(ftpcolat, ftpR, ftpX, ftpY)

        # Trim the image
        x0, x1 = int(ftpX-80), int(ftpX+80)           # [px]
        y0, y1 = int(ftpY-80), int(ftpY+80)           # [px]
        # print('x0,x1,y0,y1', x0,x1,y0,y1)
        img = dimage[y0:y1,x0:x1]
        # fig, ax = plt.subplots(dpi=100)
        # ax.pcolormesh(img)
        # plt.show()
        
        # Stats
        # mean = np.average(img)
        # median = np.median(img)
        # std = np.std(img)
        # thres = 1.0*std
        # daofind = DAOStarFinder(fwhm=2.0, threshold=thres)  
        # https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html
        # DAOStarFinderのfluxの定義 ... stdのfactor倍
        # sources = daofind(img)  

        # positions = np.transpose((sources['xcentroid'], sources['ycentroid']))  # [px]
        # positions[:,0] += x0    # [px]
        # positions[:,1] += y0    # [px]
        positions = np.array([ftpX, ftpY])

        poslat, poswlong = px2lat_wlong(positions[0], positions[1], Npx, extdeg, hem)

        title, _ = fits30s[j].split('_stis_')
        title = title+'_efp'

        """
        # Plot
        fig, ax = plt.subplots(dpi=120)
        ax.set_title(title, weight='bold')
        ax.set_aspect('equal')
        ax.set_xlim(x0-100, x1+100)
        ax.set_ylim(y0-100, y1+100)
        ax.pcolormesh(dimage, cmap=TScmap().MIDNIGHTS(),
                    norm=colors.LogNorm(vmin=10, vmax=2000),
                    zorder=0.1)
        ax.plot(ftpX, ftpY,
                markersize=7, marker='+', markerfacecolor='#f24875',
                markeredgecolor='#f24875', markeredgewidth=1, zorder=1)
        # apertures = CircularAperture(positions[posarg0,:], r=4.)
        # apertures.plot(color='red', lw=1.0, alpha=0.6)
        """
        aper, rawflux = Aperture(dimage, positions)
        # aper.plot(color='red', lw=1.0, alpha=0.6)

        """
        # Model positions
        if hem=='north':
            model_fp = model_fpN
        else:
            model_fp = model_fpS
        for i in satellite_index:
            latidx = 2*i+1
            s3idx = latidx+1
            mfpX, mfpY = lat_wlong2px(model_fp[:-1, latidx], model_fp[:-1, s3idx], Npx, extdeg, hem)
            ax.plot(mfpX, mfpY, 'y', lw=0.75, zorder=0.5)
            plt.show()
        """

        # TFindex = np.ones(positions.shape[0], dtype=bool)
        # TFindex[posarg0] = False
        # positions = positions[TFindex]

        print('=== The spot ===')
        print('Nearest pixel position [px]:', positions)
        # print('Number of pixels [px]: ', sources['npix'][0])
        print('Brightness ave [kR]: ', rawflux['final_phot_ave'][0])
        print('Brightness bkg [kR]: ', rawflux['annulus_median'][0])
        # print('Wlong [deg]', poswlong)
        # print('Lat [deg]', poslat)

        brightlist[j] = [h.datetime, poslat, poswlong, h.s3moon, rawflux['final_phot_ave'][0], rawflux['annulus_median'][0]]

        # fig.savefig(savedir+'/'+title+'.jpg')
        
    
    df = pd.DataFrame(brightlist)
    df.to_csv(savedir+'/brightness.csv', header=brightnessCSV, sep='\t')

data/red_bkg/2014/016_v12/jup_14-016-00-03-33_0030_v12_stis_f25srf2_flatproj.fits
OBS DATE 2014-01-16T00:03:33
=== JRM33 Estimation ===
Wlong [deg] 173.29767057780535
Lat [deg] 53.00263077991321
=== The spot ===
Nearest pixel position [px]: [547.13512197  98.89856327]
Brightness ave [kR]:  -27.883219922245644
Brightness bkg [kR]:  96.61146545410156
data/red_bkg/2014/016_v12/jup_14-016-00-03-43_0030_v12_stis_f25srf2_flatproj.fits
OBS DATE 2014-01-16T00:03:43
=== JRM33 Estimation ===
Wlong [deg] 173.33671215485836
Lat [deg] 53.00904857340138
=== The spot ===
Nearest pixel position [px]: [546.85307647  98.94120165]
Brightness ave [kR]:  -11.920257244111445
Brightness bkg [kR]:  74.87091827392578
data/red_bkg/2014/016_v12/jup_14-016-00-03-53_0030_v12_stis_f25srf2_flatproj.fits
OBS DATE 2014-01-16T00:03:53
=== JRM33 Estimation ===
Wlong [deg] 173.37575372532848
Lat [deg] 53.01546636580743
=== The spot ===
Nearest pixel position [px]: [546.57111134  98.98403533]
Brightness ave [kR]:  -30.134